In [ ]:
# https://www.usharbors.com/harbor/maine/winter-harbor-me/tides/?tide=2023-07#monthly-tide-chart

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
def get_tides(year, month, state, location):
    driver = webdriver.Safari()
    url = "https://www.usharbors.com/harbor/" + state + "/" + location + "/tides/?tide=" + year + "-" + month
    driver.get(url)
    df = pd.read_html(driver.page_source)[0]
    df = df.iloc[:, :-9]
    df.columns = ["date_index", "date_week", "high_am", "high_am_ft", "high_pm", "high_pm_ft"]
    df = df.iloc[:-2]
    df.loc[:, 'year'] = year
    df.loc[:, 'month'] = month
    df.loc[:,"datetime_am"] = pd.to_datetime(df.iloc[:,7] +"/"+ df.iloc[:,0] + "/" + df.iloc[:,6] +" "+
                                         df.iloc[:,2], format = "%m/%d/%Y %I:%M")
    df.loc[:,"datetime_pm"] = pd.to_datetime(df.iloc[:,7] +"/"+ df.iloc[:,0] + "/" + df.iloc[:,6] +" "+
                                             df.iloc[:,4], format = "%m/%d/%Y %I:%M") + timedelta(hours = 12)
    df['date_index'] = df['date_index'].astype('Int64')
    df = pd.melt(df, id_vars=['date_index', 'year', 'month'], value_vars=['datetime_am', 'datetime_pm'])
    df = df.sort_values(by=['date_index'])
    df.loc[:,"start_water"] = df.loc[:,"value"] + timedelta(hours = -2)
    df.loc[:,"end_water"] = df.loc[:,"value"] + timedelta(hours = 2)
    driver.close()
    return(df)

In [16]:
tides_2023_03 = get_tides("2023", "07", "maine", "winter-harbor-me")

In [17]:
tides_2023_03.to_csv('7-23-tides-winter-harbor.csv')